In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 데이터 추출 및 훈련 데이터 생성

In [4]:
import pandas as pd

train_file = pd.read_csv('/content/drive/MyDrive/captum/ratings_train.txt', sep='\t')

In [5]:
train_file.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


pandas에서 행을 반복으로 출력하는 방법 `df.iterrows()`

In [6]:
train_pair = [(row[1], row[2]) for _, row in train_file.iterrows() if type(row[1]) == str]

train_data = [pair[0] for pair in train_pair]
label_data = [pair[1] for pair in train_pair]

In [8]:
for data, label in zip(train_data[:3], label_data[:3]):
  print(f'문장: {data}\n라벨: {label}')

문장: 아 더빙.. 진짜 짜증나네요 목소리
라벨: 0
문장: 흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
라벨: 1
문장: 너무재밓었다그래서보는것을추천한다
라벨: 0


# Tokenizer 학습

In [10]:
params = {
    'batch_size': 64,
    'num_epoch': 15,
    'lr': 0.003,
    'dropout': 0.5,
    'min_frequency': 3,
    'max_len': 20,

    'vocab_size': 20000,
    'embed_dim': 100,
    'hidden_dim': 256,
    'filter_sizes': [2, 3, 4],
    'num_filters': 100,
    'output_dim': 1,
}

In [11]:
with open('train_tokenizer.txt', 'w', encoding='utf-8') as f:
  for line in train_data:
    print(line, file=f)

In [13]:
pip install tokenizers

     |████████████████████████████████| 6.8 MB 10.1 MB/s 


In [14]:
from tokenizers import BertWordPieceTokenizer

tokenizer = BertWordPieceTokenizer()

# 앞 코드에서 제작한 텍스트 파일로 토크나이저 훈련
trainer = tokenizer.train(
    files = ['train_tokenizer.txt'],
    vocab_size = params['vocab_size'],
    min_frequency = params['min_frequency'],
    special_tokens = ['[PAD]','[SOS]', '[EOS]','[UNK]'],
    wordpieces_prefix='##'
)

훈련시킨 토크나이저 확인

In [15]:
tokenizer._parameters

{'clean_text': True,
 'cls_token': '[CLS]',
 'handle_chinese_chars': True,
 'lowercase': True,
 'mask_token': '[MASK]',
 'model': 'BertWordPiece',
 'pad_token': '[PAD]',
 'sep_token': '[SEP]',
 'strip_accents': None,
 'unk_token': '[UNK]',
 'wordpieces_prefix': '##'}

스페셜 토큰들의 id값을 변수로 저장

In [16]:
pad_idx = tokenizer.token_to_id('[PAD]')
print(f'PAD_idx: {pad_idx}')
sos_idx = tokenizer.token_to_id('[SOS]')
print(f'SOS_idx: {sos_idx}')
eos_idx = tokenizer.token_to_id('[EOS]')
print(f'EOS_idx: {eos_idx}')

PAD_idx: 0
SOS_idx: 1
EOS_idx: 2


 토크나이저에 패딩 옵션 부여

In [18]:
tokenizer.enable_padding(pad_id=pad_idx, pad_token='[PAD]',length=params['max_len'])

전체 데이터셋에 토크나이즈 수행

In [19]:
encoded_data = tokenizer.encode_batch(train_data)

In [20]:
print(f'train data: {len(train_data)}개')
print(f'label_data: {len(label_data)}개')
print(f'encoded_data: {len(encoded_data)}개')

train data: 149995개
label_data: 149995개
encoded_data: 149995개


토크나이즈 수행 결과 확인

In [21]:
print(f'토큰: {encoded_data[216].tokens}\n')
print(f'아이디: {encoded_data[216].ids}')

토큰: ['아', '너무', '웃기고', '배꼽', '빠질', '##뻔', '##했네', '^', '^', '내', '컴', '##에', '이', '영화', '있는데', '^', '^', '[PAD]', '[PAD]', '[PAD]']

아이디: [829, 906, 4451, 7918, 8742, 2153, 4946, 39, 39, 905, 5053, 819, 809, 821, 2990, 39, 39, 0, 0, 0]


# 토크나이즈 결과 후처리
1. 문장 시작을 알리는 'SOS'토큰 추가
2. 최대 길이를 넘는 문장들을 모두 최대길이에 맞게 슬라이스 연산
3. 문장이 짧아 패딩 토큰이 포함된 문장은 첫 번째 패딩 토큰을 'EOS'토큰으로 변경
4. 패딩 토큰이 포함되지 않은 문장은 시퀀스 말미에 'EOS'토큰 삽입

`SOS와 EOS 토큰은 단순 분류 모델에서는 사용하지 않아도 무방`

In [22]:
def postprocess(input_ids):
  input_ids = [sos_idx] + input_ids # 1. 문장 시작을 알리는 'SOS'토큰 추가

  input_ids = input_ids[:params['max_len']] # 2. 슬라이스

  # 3.
  if pad_idx in input_ids:
    pad_start = input_ids.index(pad_idx)
    input_ids[pad_start] = eos_idx
  
  #4.
  else:
    input_ids[-1] = eos_idx
  return input_ids

In [23]:
processed_data = [postprocess(data.ids) for data in encoded_data]

In [26]:
print(f'후처리 결과 아이디: {processed_data[216]}')
print(f'후처리 결과 디코딩: {tokenizer.decode(processed_data[216])}')

후처리 결과 아이디: [1, 829, 906, 4451, 7918, 8742, 2153, 4946, 39, 39, 905, 5053, 819, 809, 821, 2990, 39, 39, 2, 0]
후처리 결과 디코딩: 아 너무 웃기고 배꼽 빠질뻔했네 ^ ^ 내 컴에 이 영화 있는데 ^ ^


# CNN classifier

In [37]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torch.utils.data import DataLoader

torch.manual_seed(32)
torch.cuda.manual_seed(32)

device = torch.device("cuda" if torch.cuda.is_available() else 'cpu')

In [28]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.embedding = nn.Embedding(params['vocab_size'], params['embed_dim'], padding_idx=pad_idx)

        self.convs = nn.ModuleList([nn.Conv2d(in_channels=1, 
                                              out_channels=params['num_filters'], 
                                              kernel_size=(fs, params['embed_dim'])) 
                                    for fs in params['filter_sizes']])
        
        self.fc = nn.Linear(len(params['filter_sizes']) * params['num_filters'], 1)
        
        self.dropout = nn.Dropout(params['dropout'])
        
    def forward(self, input_ids):
        # input_ids    = [배치 사이즈, 문장 길이]

        embedded = self.embedding(input_ids).unsqueeze(1)
        # embedded     = [배치 사이즈, 채널 개수, 임베딩 차원]
        
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
        # conved_n     = [배치 사이즈, 필터 개수, 문장 길이 - 필터 리스트[n] + 1]
        
        max_pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        # max_pooled_n = [배치 사이즈, 필터 개수]

        cat = self.dropout(torch.cat(max_pooled, dim = 1))
        # cat          = [배치 사이즈, 필터 개수 x len(필터 리스트)]

        return self.fc(cat)  # [배치 사이즈, 1]

# 데이터 텐서 변환

In [30]:
processed_data = [torch.LongTensor(data).to(device) for data in processed_data]
train_label = [torch.FloatTensor([label]).to(device) for label in label_data]

In [35]:
processed_data[0]

tensor([    1,   829,  2463,    17,    17,   954, 16355,  2862,     2,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0])

In [36]:
train_label[0]

tensor([0.])

In [38]:
train_iter = DataLoader(processed_data, batch_size=params['batch_size'])
label_iter = DataLoader(train_label, batch_size=params['batch_size'])

# 모델 훈련

In [40]:
model = CNN()
model.to(device)

criterion = nn.BCEWithLogitsLoss()
criterion.to(device)

optimizer = optim.Adam(model.parameters(), lr=params['lr'])

for epoch in range(params['num_epoch']):
  model.train()
  epoch_loss = 0

  for (batch, label) in zip(train_iter, label_iter):
    optimizer.zero_grad()

    logits = model(batch).squeeze(1)
    labels = label.view(label.size(0))

    loss = criterion(logits, labels)
    epoch_loss += loss.item()

    loss.backward()
    optimizer.step()

  train_loss = epoch_loss / len(train_iter)
  print(f'Epoch: {epoch+1:02} | Train_loss: {train_loss:.3f}')


Epoch: 01 | Train_loss: 0.477
Epoch: 02 | Train_loss: 0.353
Epoch: 03 | Train_loss: 0.312
Epoch: 04 | Train_loss: 0.280
Epoch: 05 | Train_loss: 0.246
Epoch: 06 | Train_loss: 0.214
Epoch: 07 | Train_loss: 0.189
Epoch: 08 | Train_loss: 0.171
Epoch: 09 | Train_loss: 0.153
Epoch: 10 | Train_loss: 0.139
Epoch: 11 | Train_loss: 0.128
Epoch: 12 | Train_loss: 0.117
Epoch: 13 | Train_loss: 0.113
Epoch: 14 | Train_loss: 0.108
Epoch: 15 | Train_loss: 0.103


# Captum을 이용한 모델 해석

In [43]:
pip install captum

     |████████████████████████████████| 1.4 MB 8.8 MB/s 


In [44]:
from captum.attr import LayerIntegratedGradients, TokenReferenceBase, visualization

In [45]:
token_reference = TokenReferenceBase(reference_token_idx=pad_idx) # 레퍼런스 생성을 위한 모듈
lig = LayerIntegratedGradients(model, model.embedding) # 결과 해석에 사용되는 integratedGradient 기법 모듈

* 이제 결과 분석을 위해 비교 문장을 생성할 interpret_sentence 함수와,
* 각 결과를 VisualziationDataRecord 클래스 인스턴스로 저장해 줄 add_attributions_to_visualizer 함수를 정의합니다.

In [46]:
vis_data_records_ig = []

label_vocab  = {0: '부정', 1: '긍정'}

def interpret_sentence(model, sentence, label = 0):       
    model.zero_grad()
    
    input_ids = tokenizer.encode(sentence)
    input_tokens = input_ids.tokens[:params['max_len']]
    
    input_ids = postprocess(input_ids.ids)
    input_indices_tensor = torch.LongTensor(input_ids).to(device).unsqueeze(0)

    # 단일 문장에 대한 예측 작업 수행
    pred = torch.sigmoid(model(input_indices_tensor)).item()
    pred_ind = round(pred)

    # 베이스 라인 역할을 할 Reference 생성: 주로 패딩 토큰으로 채워줌
    reference_indices = token_reference.generate_reference(params['max_len'], device=device).unsqueeze(0)

    # LayerIntegratedGradients 모듈 활용해 개별 단어의 속성값 및 델타값 근사치 계산
    attributions_ig, delta = lig.attribute(input_indices_tensor, reference_indices, n_steps=500, return_convergence_delta=True)

    print('pred: ', label_vocab[pred_ind], '(', '%.2f'%pred, ')', ', delta: ', abs(delta))

    add_attributions_to_visualizer(attributions_ig, input_tokens, pred, pred_ind, label, delta, vis_data_records_ig)
    

def add_attributions_to_visualizer(attributions, text, pred, pred_ind, label, delta, vis_data_records):
    attributions = attributions.sum(dim=2).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    attributions = attributions.cpu().detach().numpy()

    # 시각화 위해 샘플을 리스트에 추가
    vis_data_records.append(visualization.
                                VisualizationDataRecord(
                                    attributions,
                                    pred,
                                    label_vocab[pred_ind],
                                    label_vocab[label],
                                    label_vocab[1],
                                    attributions.sum(),       
                                    text,
                                delta
                                )
                           )

In [47]:
# 예제 문장 추가 및 분석 수행
interpret_sentence(model, '간만에 쫄깃한 추리 수사극을 볼수 있었습니다.명 배우들의 연기에 시간 가는 줄 몰랐네요.올해 본 영화중 가장 만족도 높은 영화였습니다.', label=1)
interpret_sentence(model, '평이 워낙 좋아 갔는데 거품이 심한듯.. 개연성도 엉망에 르부아 블랑의 뜬금없는 직관에만 의존한 스토리 전개 ;; 평점 거품이 심한듯', label=0)
interpret_sentence(model, '너무 재미있게 봤습니다. 클래식한 추리소설 느낌.타 인기영화때문에 개봉관이 적어 아쉽네요', label=1)
interpret_sentence(model, '아, 2시간 넘게 지루하고 어이없는 추격전과 의욕이 넘쳐 과욕으로 변한 연기, 게다가 예상보다 너무 뻔한 결말로 고문당함.', label=0)

pred:  긍정 ( 1.00 ) , delta:  tensor([19.7103], dtype=torch.float64)
pred:  부정 ( 0.00 ) , delta:  tensor([39.8031], dtype=torch.float64)
pred:  긍정 ( 1.00 ) , delta:  tensor([1.4548], dtype=torch.float64)
pred:  부정 ( 0.00 ) , delta:  tensor([10.0312], dtype=torch.float64)


In [48]:
# 시각화 결과 표 변환
visualization.visualize_text(vis_data_records_ig)

True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
긍정,긍정 (1.00),긍정,0.97,간만에 쫄기 ##ᆺ한 추리 수사 ##극을 볼수 있었습니다 . 명 배우들의 연기에 시간 가는 줄 몰랐네 ##요 . 올해 본
부정,부정 (0.00),긍정,-1.23,평이 워낙 좋아 갔는데 거품이 심한 ##듯 . . 개연성도 엉망 ##에 르 ##부 ##아 블 ##랑 ##의 뜬금없는 직
긍정,긍정 (1.00),긍정,0.82,너무 재미있게 봤습니다 . 클래식 ##한 추리 ##소설 느낌 . 타 인기 ##영화 ##때문에 개봉 ##관이 적어 아쉽네요 [PAD] [PAD]
부정,부정 (0.00),긍정,-1.94,"아 , 2시간 넘게 지루하고 어이없는 추격 ##전과 의 ##욕이 넘쳐 과 ##욕 ##으로 변 ##한 연기 , 게다가 예상"


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
긍정,긍정 (1.00),긍정,0.97,간만에 쫄기 ##ᆺ한 추리 수사 ##극을 볼수 있었습니다 . 명 배우들의 연기에 시간 가는 줄 몰랐네 ##요 . 올해 본
부정,부정 (0.00),긍정,-1.23,평이 워낙 좋아 갔는데 거품이 심한 ##듯 . . 개연성도 엉망 ##에 르 ##부 ##아 블 ##랑 ##의 뜬금없는 직
긍정,긍정 (1.00),긍정,0.82,너무 재미있게 봤습니다 . 클래식 ##한 추리 ##소설 느낌 . 타 인기 ##영화 ##때문에 개봉 ##관이 적어 아쉽네요 [PAD] [PAD]
부정,부정 (0.00),긍정,-1.94,"아 , 2시간 넘게 지루하고 어이없는 추격 ##전과 의 ##욕이 넘쳐 과 ##욕 ##으로 변 ##한 연기 , 게다가 예상"
